In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_vendor 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH asm_mb AS (
  SELECT * FROM (
    SELECT ROW_NUMBER() OVER (PARTITION BY product_id, end_date ORDER BY proc_date) rn, *
    FROM {catalog_name}.udp_wcm_silver_dim.stg_vmp_assortmentmb
  ) WHERE rn=1
)
, asm_mn AS (
  SELECT * FROM (
    SELECT ROW_NUMBER() OVER (PARTITION BY product_id, end_date ORDER BY proc_date) rn, *
    FROM {catalog_name}.udp_wcm_silver_dim.stg_vmp_assortmentmn
  ) WHERE rn=1
)
SELECT
    b.store_id,
    product_id,
    RIGHT(CONCAT('000000000', vendor_id), 10) vendor_id,
    vendor_name,
    start_date,
    end_date
FROM asm_mb a,{catalog_name}.udp_wcm_gold_vcm_dwh.d_store b 
WHERE b.region_domain = 'North' AND b.business_unit = '2000'
UNION ALL
SELECT
    b.store_id,
    product_id,
    RIGHT(CONCAT('000000000', vendor_id), 10) vendor_id,
    vendor_name,
    start_date,
    end_date
FROM asm_mn a, {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b 
WHERE b.region_domain <> 'North' AND business_unit = '2000';
""")